In [89]:
import pickle
import os
import random
import sqlite3
import random
import numpy 

In [144]:
conn = sqlite3.connect('yelp_dataset.db')
c = conn.cursor()

def Select(q):
    return c.execute(q)

def Update(q):
    c.execute(q)
    conn.commit()

In [148]:
import gzip
user_item = set()
with gzip.open("train_data.txt.gz",'rb') as f:
    for line in f:
        user_id,item_id,rating,text = line.split("\t")
        user_item.add(user_id+"-"+item_id)
    

In [149]:
q=Select("select user_id, business_id, review_id,  starts from Reviews ")

In [150]:
user_review_dict = {}
item_review_dict = {}

In [151]:
i=0
review_rating_pairs=set()
for row in q:
    if i % 100000 == 0:
        print i
    user = row[0]
    item = row[1]
    review_id = row[2]
    rating = row[3]
    if user+"-"+item in user_item:
        if user not in user_review_dict:
            user_review_dict[user] = []
        if item not in item_review_dict:
            item_review_dict[item] = []
        user_review_dict[user] += [(review_id,rating)]
        item_review_dict[item] += [(review_id,rating)]
        review_rating_pairs.add((review_id,rating))
    i += 1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000


In [41]:
pickle.dump([user_review_dict,item_review_dict],open("user_item_rev_rating.pkl","wb"))

In [152]:
del user_item
conn.close()

In [153]:
len(list(user_review_dict.keys()))

1050896

In [154]:
threshold=11

In [155]:
len(review_rating_pairs)

3789518

In [175]:
user_review_sample={}
item_review_sample={}

In [ ]:
j=0
for u in user_review_dict:
    if j % 1000==0:
        print j,
    reviews=user_review_dict[u]
    if threshold-len(reviews)>0:
        num=threshold-len(reviews)
        sample=random.sample(review_rating_pairs,num) 
        reviews.extend(sample)
    else:
        reviews=random.sample(reviews,threshold) 
    reviews=sorted(reviews, key=lambda x: x[1])
    user_review_sample[u]=[r[0] for r in reviews]  
    j += 1

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000

In [171]:
user_review_sample.items()[:2]

(u'CM4oH2tM7g34LKXY8y8bpw',
 [u'VX-A-dvKIBVXxW32RIOlwA',
  u'ut-XZDDGNsHcL0SKz2zYCA',
  u'9u1Z7PFiTmW4BvqLtnaVvg',
  u'Ff3P6CMYH1VhsGtjU3sIRg',
  u'z5CvTFrRXUWkvN09JgmHPQ',
  u'ZrRt9i4pNlSZFtHQDBHjvw',
  u'yJEc0P4p1ioc1nXP86tfrg',
  u'ykGR4sTkZJrn2Ljk7gjaaA',
  u'M3NPCPHY3KNrKsmAM8EnZQ',
  u'ptVGvWrDrthDife7xlSPUw',
  u'-vEvg0DPf-I9XRxaM1HJpA'])

In [ ]:
k=0
for i in item_review_dict:
    if k % 10000==0:
        print k,
    reviews=item_review_dict[i]
    if threshold-len(reviews)>0:
        num=threshold-len(reviews)
        sample=random.sample(review_rating_pairs,num) 
        reviews.extend(sample)
    else:
        reviews=reviews[:threshold]
    reviews=sorted(reviews, key=lambda x: x[1])
    item_review_sample[i]=[r[0] for r in reviews]  
    k+=1

In [ ]:
item_review_sample.items()[:2]

In [ ]:
pickle.dump([user_review_sample,item_review_sample],open("user_item_rev_samples.pkl","wb"))

In [85]:
class ReviewProcessing:
    def __init__(self):
        self.review_dict = {}
        
    def load(self,review_delim_path,review_emb_path):
        self.review_dict = pickle.load(open(review_delim_path,"rb"))
        self.review_emb_path = review_emb_path
    
    def get_emb(self,review_id):
        if not self.review_dict:
            raise Exception("load the word index and emb files")
        if not review_id in self.review_dict:
            return None
        start,length = self.review_dict[review_id]
        with open(self.review_emb_path,"r") as f:
            f.seek(start)
            line = f.read(length)
            emb = line.split("\t")[4]
            emb = numpy.array([float(x) for x in emb.split()])
        return emb
    

In [86]:
rp = ReviewProcessing()
rp.load("review_delim.pkl","review_emb.txt")